Durante essa aula vamos explorar um pouco os tipos de tarefas que podemos resolver utilizando os conceitos aprendidos nas últimas 2 aulas.

A primeira tarefa será de **classificacão de texto**, similar à tarefa explorada na primeira semana.

A tarefa consiste em analisar uma notícia e classificá-la dentre um conjunto possível de temas, por exemplo, *tecnologia*, *esporte*, etc.

Para isto, vamos utilizar o dataset liberado pela [BBC](http://mlg.ucd.ie/datasets/bbc.html).



In [ ]:
import pandas as pd

data = pd.read_csv("data/text_classification/bbc-text.csv")

num_classes = len(data.category.value_counts())


training_proportion = 0.9

training_index = int(training_proportion * len(data))

train_data = data.iloc[:training_index,:]
test_data = data.iloc[training_index:, :]

train_text = train_data.loc[:, 'text'].tolist()

data.head()

Nas últimas aulas, apesar de utilizarmos embeddings em todas as tarefas, estávamos os aprendendo do zero baseando-se na base de treinamento.

Na verdade, não há necessidade disso porque já existe uma variedade muito grande de embeddings pré-treinados que podem ser utilizados.

Para este exercício vamos utilizar o [GloVe](https://nlp.stanford.edu/projects/glove/), mas existem muitas opcões de embeddings pré-treinados, inclusive em português.

Antes de rodar as próximas linhas, baixe o arquivo [glove.6B.50d.txt](https://github.com/uclnlp/inferbeddings/blob/master/data/glove/glove.6B.50d.txt.gz)

Vamos ler o conteúdo do arquivo com embeddings pré-treinados, para posteriormente utilizá-los na nossa camada de Embedding. O propósito da próxima célula é determinar o vocabulário para a nossa base de treinamento, e carregar os embeddings para todas as palavras encontradas.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

max_vocab = 2000

tokenizer = Tokenizer(num_words = max_vocab)
tokenizer.fit_on_texts(train_text)

indexes = min(max_vocab, len(tokenizer.word_index))

num_emb = 0
cons_keys = []
for word in tokenizer.word_index.keys():
    cons_keys.append(word)
    num_emb += 1
    if num_emb == max_vocab:
        break


embeddings_index = dict()
f = open('data/text_classification/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    if word in cons_keys:
        coefs = np.array(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((max_vocab, 50))
for word, i in tokenizer.word_index.items():
    if i>=max_vocab:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
print(embedding_matrix[10:13])


Agora faca o processamento restante necessário, including padding.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

sentence_size = # Seu parametro

dataset_train_sequences = #TOkenize a base de treinamento 
dataset_test_sequences = #TOkenize a base de teste

padded_train = # Realize Padding na base de treinamento
padded_test = # Realize Padding na base de teste




In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM
from tensorflow.keras import Sequential


model = Sequential()
# 50 é o número de dimensoes do embedding (definido pelo arquivo que baixamos)
#trainable = False significa que esses pesos nao sao atualizados durante o treinamento.
model.add(Embedding(max_vocab, 50, weights=[embedding_matrix], input_length=sentence_size, trainable=False))
#Resto da sua arquitetura

optimizer = # otimizador
loss = #Loss

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.utils as ku
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
labelencoder.fit_transform(train_data.loc[:, 'category'])

label_train = train_data.loc[:, ['category']].apply(LabelEncoder().fit_transform).values
label_test = test_data.loc[:, ['category']].apply(LabelEncoder().fit_transform).values

label_train = ku.to_categorical(label_train, num_classes = num_classes)
label_test = ku.to_categorical(label_test, num_classes = num_classes)


In [ ]:
num_epochs = 10
model.fit(padded_train,label_train, epochs = num_epochs, validation_data=(padded_test,label_test), verbose=2 )

In [ ]:
#Implemente essa funcao para pegar uma lista de textos e retornar o assunto de cada um
def predict_topic(text):
    #Pre processamento
    topic = np.argmax(model.predict(proc_text), axis=1)
    topic = labelencoder.inverse_transform(topic)
    return topic

predict_topic(['A lot of data centers',"A lot of stock prices and financial data", 'A lot of goals scored'])
    

Agora que já somos experts em classificar sentimentos em textos, vamos considerar uma tarefa diferente mas igualmente importante: **Traducão**.

Como realizar traducões entre duas linguas requer modelos grandes e um grande corpus de treinamento, vamos considerar uma tarefa de "traducão" mais simples.

A tarefa que consideraremos é a traducão de uma data em texto livre para um formato mais palatável para o computador. Alguns exemplos de traducão estão abaixo:

`9 may 1998 -> 1998-05-09
10.11.19 -> 2019-11-10
9/10/70 -> 1970-09-10
saturday april 28 1990 -> 1990-04-28
thursday january 26 1995 -> 1995-01-26
monday march 7 1983 -> 1983-03-07`

Para realizar esta tarefa, modelaremos o problema de uma forma um pouco diferente. Para a criacão de nosso vocabulário, designaremos um código para cada caractere (não cada palavra), formando um vocabulario de origem e um de destino.

Seu modelo irá ler um conjunto de caracteres de entrada de tamanho `n`, processá-lo e dar como saída um conjunto de caracteres de tamanho `10`.



In [ ]:
data = pd.read_csv('data/translation/dates_dataset.csv', header=None)
train_index = int(0.9 * len(data))
train_data = data[:train_index]
test_data = data[train_index:]

#Construindo o tokenizer para cada caracter com o parametro char_level = True
tok_origem = Tokenizer(char_level=True)
tok_destino = Tokenizer(char_level = True)

tok_origem.fit_on_texts(train_data.loc[:,0])
tok_destino.fit_on_texts(train_data.loc[:,1])

seq_train = tok_origem.texts_to_sequences(train_data.loc[:,0])
seq_test = tok_origem.texts_to_sequences(test_data.loc[:,0])

label_train = np.array(tok_destino.texts_to_sequences(train_data.loc[:,1]))
label_test = np.array(tok_destino.texts_to_sequences(test_data.loc[:,1]))

label_train = ku.to_categorical(label_train, num_classes = len(tok_destino.word_index)+1)
label_test = ku.to_categorical(label_test, num_classes = len(tok_destino.word_index)+1)



In [ ]:
max_size_orig = 30
max_size_dest = 10

padded_train = pad_sequences(seq_train, maxlen=max_size_orig, padding='post', truncating='post')
padded_test = pad_sequences(seq_test, maxlen=max_size_orig, padding='post', truncating='post')



In [ ]:
from tensorflow.keras.layers import Reshape, Lambda, Dropout
import tensorflow.keras.backend as K

model = Sequential()
model.add(Embedding(len(tok_origem.word_index)+1, 50,  input_length=max_size_orig))
#Sua Arquitetura
model.add(Dense(len(tok_destino.word_index)+1, activation='softmax'))


optimizer = #Optimizer
loss = #Loss

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs = 50

model.fit(padded_train,label_train, epochs = num_epochs, validation_data=(padded_test,label_test), verbose=2 )

In [ ]:
#Fazer uma funcao que pega uma data no formato de origem e a transforma para o formato destino
def predict_dates(dates):
    #Seu codigo aqui
    return converted
                           
predict_dates(['august 26 1975', 'friday july 12 1991', '10 sep 1975'])


Para o último problema a ser resolvido, vamos considerar um problema um pouco mais exótico, semelhante ao problema de geracão de palavras estudado na segunda aula.


Para esta tarefa geraremos **nomes de Dinossauros**.

Nomes de dinossauros têm uma certa semelhanca, por exemplo:

`
Abdallahsaurus
Abelisaurus
Abrictosaurus
`

Portanto, vamos treinar um modelo que completa o nome de um dinossauro!

Para isto, novamente vamos gerar um token para cara caractere. O modelo deve ser treinado recebendo um nome parcial e o completando, por exemplo:

`x = 'Abda', y = 'Abdallahsaurus'
x = 'Abelisa', y = 'Abelisaurus'
`

O modelo pode ser treinado com a base de treinamento sendo gerada similarmente ao problema de gerar sonetos, mas adicionando um token "\n" à saída para indicar o final do nome do Dinossauro:


`x = 'A', y = 'Aardonyx\n'
x = 'Aa', y = 'Aardonyx\n'
x = 'Aar', y = 'Aardonyx\n'
...
x = 'Aardony', y = 'Aardonyx\n'
`



In [ ]:
row_list =[]
largest_name = 0
with open("data/text_generation/dinos.txt", "r") as a_file:
  for line in a_file:
    largest_name = max(largest_name, len(line))
    for i in range(len(line)-1):
        row_list.append([line[:i+1], line])

data = pd.DataFrame(row_list, columns = ['x', 'y'])
data.head(12)
        

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)
training_index = int(0.9*len(data))
train_data = data.loc[:training_index,:]
test_data = data.loc[training_index:, :]

#Faća agora todo o processo de pre-processamento ate o treinamento do modelo

In [ ]:
model = Sequential()
model.add(Embedding(words_num+1, 50,  input_length=largest_name))
#Sua arquitetura
model.add(Dense(words_num+1, activation='softmax'))


optimizer = #Optimizer
loss = #Loss

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.summary()


In [ ]:
num_epochs = 5

model.fit(padded_train_seq,train_labels, epochs = num_epochs, validation_data=(padded_test_seq,test_labels), verbose=2 )

In [ ]:
def predict_dino(prefixos):
    seqs = tok.texts_to_sequences(prefixos)
    padded = pad_sequences(seqs, maxlen=largest_name, padding='post', truncating='post')
    predictions = model.predict(padded)
    predictions = np.argmax(predictions,axis=2)
    converted = tok.sequences_to_texts(predictions)
    converted = ["".join(x.split()) for x in converted]
    final = [x.split('\n')[0] for x in converted]
    final = [prefixos[i]+final[i][len(prefixos[i]):] for i in range(len(final))]
    return final
             
predict_dino(['test', 'brasil', 'fiep'])                        